<a href="https://colab.research.google.com/github/Almamun809/Daily-NLP/blob/main/Basic_News_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

#dataset
news_data = [
    ("The government of Bangladesh has made improvements in education.", "Politics"),
    ("Cricket is the most popular sport in Bangladesh.", "Sports"),
    ("The stock market in Bangladesh saw significant growth last year.", "Business"),
    ("There are new educational reforms to help students in Bangladesh.", "Politics"),
    ("Bangladesh cricket team won a major international match.", "Sports")
]

texts, labels = zip(*news_data)

vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(texts).toarray()

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Neural Network
class NewsClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NewsClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

input_dim = X_train.shape[1]
hidden_dim = 64
output_dim = len(label_encoder.classes_)
learning_rate = 0.001
epochs = 20

model = NewsClassifier(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()

    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Evaluation
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

    accuracy = accuracy_score(y_test_tensor, predicted)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    precision = precision_score(y_test_tensor, predicted, average='weighted')
    print(f"Precision: {precision:.2f}")

    recall = recall_score(y_test_tensor, predicted, average='weighted')
    print(f"Recall: {recall:.2f}")

    f1 = f1_score(y_test_tensor, predicted, average='weighted')
    print(f"F1-Score: {f1:.2f}")

    conf_matrix = confusion_matrix(y_test_tensor, predicted)
    print("Confusion Matrix:")
    print(conf_matrix)